In [0]:
%sql
SELECT current_user();


current_user()
fabtrainer@shalinigupta2501outlook.onmicrosoft.com


In [0]:
%sql
SHOW STORAGE CREDENTIALS;


name,comment
adls_ext_azuremanagedidentity_1762860583352,null
dbworkspace,null
fab_az_databricks_ws,null


In [0]:
df = spark.read.load('abfss://datacontiner@storage2fordb.dfs.core.windows.net/Baby_Names__Beginning_2007.csv', format='csv', header=True)
display(df.limit(10))

Year,First Name,County,Sex,Count
2021,OLIVIA,Albany,F,16
2021,AMELIA,Albany,F,16
2021,SOPHIA,Albany,F,13
2021,AVA,Albany,F,13
2021,EMMA,Albany,F,12
2021,PENELOPE,Albany,F,11
2021,MADISON,Albany,F,10
2021,ABIGAIL,Albany,F,10
2021,HARPER,Albany,F,10
2021,ISLA,Albany,F,9


In [0]:
df = df.withColumnRenamed("First Name", "First_Name")
display(df.limit(10))

Year,First_Name,County,Sex,Count
2021,OLIVIA,Albany,F,16
2021,AMELIA,Albany,F,16
2021,SOPHIA,Albany,F,13
2021,AVA,Albany,F,13
2021,EMMA,Albany,F,12
2021,PENELOPE,Albany,F,11
2021,MADISON,Albany,F,10
2021,ABIGAIL,Albany,F,10
2021,HARPER,Albany,F,10
2021,ISLA,Albany,F,9


In [0]:
delta_table_path = "abfss://datacontiner@storage2fordb.dfs.core.windows.net/delta/baby-delta"
df.write.format("delta").save(delta_table_path)

In [0]:
from delta.tables import *
from pyspark.sql.functions import *
   
# Create a deltaTable object
deltaTable = DeltaTable.forPath(spark, delta_table_path)
# Update the table 
deltaTable.update(
    condition = "First_Name == 'MADISON'",
    set = {"Count":"Count + 2"})
# View the updated data as a dataframe
deltaTable.toDF().show(10)

+----+----------+-------+---+-----+
|Year|First_Name| County|Sex|Count|
+----+----------+-------+---+-----+
|2012|     ELIAS|SUFFOLK|  M|   11|
|2012|    VICTOR|SUFFOLK|  M|   11|
|2012|    KAIDEN|SUFFOLK|  M|   11|
|2012|    DAMIAN|SUFFOLK|  M|   11|
|2012|     MARCO|SUFFOLK|  M|   11|
|2012|    JAIDEN|SUFFOLK|  M|   11|
|2012|      AXEL|SUFFOLK|  M|   11|
|2012|    MATTEO|SUFFOLK|  M|   11|
|2012|   JAMESON|SUFFOLK|  M|   11|
|2012|     MARIO|SUFFOLK|  M|   11|
+----+----------+-------+---+-----+
only showing top 10 rows


In [0]:
new_df = spark.read.format("delta").load(delta_table_path)
new_df.show(10)

+----+----------+-------+---+-----+
|Year|First_Name| County|Sex|Count|
+----+----------+-------+---+-----+
|2012|     ELIAS|SUFFOLK|  M|   11|
|2012|    VICTOR|SUFFOLK|  M|   11|
|2012|    KAIDEN|SUFFOLK|  M|   11|
|2012|    DAMIAN|SUFFOLK|  M|   11|
|2012|     MARCO|SUFFOLK|  M|   11|
|2012|    JAIDEN|SUFFOLK|  M|   11|
|2012|      AXEL|SUFFOLK|  M|   11|
|2012|    MATTEO|SUFFOLK|  M|   11|
|2012|   JAMESON|SUFFOLK|  M|   11|
|2012|     MARIO|SUFFOLK|  M|   11|
+----+----------+-------+---+-----+
only showing top 10 rows


In [0]:
new_df = spark.read.format("delta").option("versionAsOf", 0).load(delta_table_path)
new_df.show(10)

+----+----------+-------+---+-----+
|Year|First_Name| County|Sex|Count|
+----+----------+-------+---+-----+
|2019|     SOFIA|Suffolk|  F|   41|
|2019|    GIANNA|Suffolk|  F|   41|
|2019|  SAMANTHA|Suffolk|  F|   41|
|2019| MACKENZIE|Suffolk|  F|   41|
|2019|    EMILIA|Suffolk|  F|   40|
|2019|   GENESIS|Suffolk|  F|   39|
|2019|      MILA|Suffolk|  F|   38|
|2019|    HAILEY|Suffolk|  F|   38|
|2019|    HARPER|Suffolk|  F|   37|
|2019|     RILEY|Suffolk|  F|   36|
+----+----------+-------+---+-----+
only showing top 10 rows


In [0]:
%sql
Use dbworkspace.default;
DROP TABLE IF EXISTS babyexternal;
CREATE TABLE babyexternal USING DELTA LOCATION "abfss://datacontiner@storage2fordb.dfs.core.windows.net/babyexternal"
    


In [0]:
%sql
DESCRIBE EXTENDED babyexternal;

col_name,data_type,comment
,,
# Detailed Table Information,,
Catalog,dbworkspace,
Database,default,
Table,babyexternal,
Created Time,Tue Nov 11 12:21:11 UTC 2025,
Last Access,UNKNOWN,
Created By,Spark,
Type,EXTERNAL,
Location,abfss://datacontiner@storage2fordb.dfs.core.windows.net/babyexternal,
